In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import docker

In [15]:
client = docker.from_env()

In [211]:
_image = "volume-nfs"
print(_image)

volume-nfs


In [212]:
old_registry = "registry.vngcloud.vn/public"
new_registry = "vcr.vngcloud.vn/81-vks-public"

In [213]:
images = client.images.pull(f"{old_registry}/{_image}", all_tags=True)

In [214]:
for image in images:
    img = image.tags[0]
    new_image = img.replace(old_registry, new_registry)
    print(image.tags[0])
    image.tag(new_image)

registry.vngcloud.vn/public/volume-nfs:0.8


In [215]:
for image in images:
    img = image.tags[0]
    new_image = img.replace(old_registry, new_registry)
    print(f"Pushing... {new_image}")
    client.images.push(new_image)
    print(f"Pushed {new_image}\n")

Pushing... vcr.vngcloud.vn/81-vks-public/volume-nfs:0.8
Pushed vcr.vngcloud.vn/81-vks-public/volume-nfs:0.8



In [216]:
!docker image prune -a -f

Deleted Images:
untagged: vcr.vngcloud.vn/81-vks-public/vcontainer-terminal:latest
untagged: registry.vngcloud.vn/public/volume-nfs:0.8
untagged: registry.vngcloud.vn/public/volume-nfs@sha256:3899ca782a272608fb4139eca436e87592eb779ae76adad6f8e0080365d57de0
untagged: vcr.vngcloud.vn/81-vks-public/volume-nfs:0.8
untagged: vcr.vngcloud.vn/81-vks-public/volume-nfs@sha256:3899ca782a272608fb4139eca436e87592eb779ae76adad6f8e0080365d57de0
deleted: sha256:ab7049d62a53be18647edab2f82e037bbc43364866d8454f95618f005ae7663e
deleted: sha256:0ed1053e034c1d8ef82730c7e89f9862d307f64b26ad7d956c41350acc65c63c
deleted: sha256:65c672f6c7dcefb45c534c0878ffc164a33313aced899e3982bb9393de2858f1
deleted: sha256:bf28240266880bb241606a211eece232c90bf1641fa8b9fae657755eeda1b387
deleted: sha256:a80fbe280ccea3c3a497815a4ec0bf57177369b87803389d9bafda162d6bd342
deleted: sha256:905c5782a3bfebd965ca2586f09abde4737870d3b0b7ab0094c8dc24701d1f02
deleted: sha256:460d95a661743d266fa9c81e6f8b6cd8e59cf09d829ebb1969c2942bcf10b39